In [ ]:
import base64
import anthropic
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()
client = anthropic.Anthropic()

In [26]:
def read_file_and_encode(file_path):
    with open(file_path, 'rb') as pdf_file:
        pdf_base64 = base64.standard_b64encode(pdf_file.read()).decode("utf-8")
    
    return pdf_base64

In [51]:
def get_section_1_2_instruction():

    return """
    ## Core Requirements
    - Use ONLY information from the provided V&V document - no external knowledge or invented details
    - Output a coherent, text-only narrative report with clear section headings
    - If information is missing, omit that section or note it as unavailable
    - Focus on stability-related content: look for keywords like "stability testing," "aging," "shelf life," 
    "long-term performance"

    ### 1. Summary (Executive Summary)
    - Brief overview of study purpose and key findings
    - Critical data points from stability testing
    - Overall conclusion on product stability status and recommended shelf life

    ### 2. Introduction
    - Purpose and rationale of the stability study
    - Product description and context from V&V
    - Scope: duration, conditions, aspects evaluated, sample sizes
    - Regulatory/standard framework if mentioned
    - Key assumptions about test conditions
    """

def get_section_3_instructions():
    return """
    ## Core Requirements
        - Use ONLY information from the provided V&V document - no external knowledge or invented details
        - Extract references from Section 3 of the V&V document
    
        - Filter to include only references relevant to stability testing and product shelf life
    ### 3. References
        - Identify all references listed in the V&V document's reference section
        - Filter and include only references that are relevant to:
        -Stability testing methodology and protocols
        -Aging studies and shelf life determination  
        -Sterilization reports that affect product stability
        -Laboratory test results related to aging/stability
        -Manufacturing documents with batch/lot traceability for stability samples
        -Risk management documents (FMEA) identifying aging-related risks
        -ECO/change control documents driving the stability study need
        - Format as a numbered reference list with Document ID and Revision
        - Exclude references that are purely design-related, marketing, or regulatory reviews not related to 
            stability
        - Include brief justification for why each reference is relevant to stability work
    """

def get_section_4_instructions():
    return """
    ## Core Requirements
    - Use ONLY information from the provided V&V document - no external knowledge or invented details
    - Extract key principles and considerations that guide the stability study
    - Focus on the rationale and decision-making framework for the study design
    
    ### 4. Guiding Considerations
    - Primary objectives of the stability study (what needs to be proven/validated)
    - Key risk factors identified that could affect product stability over time
    - Critical quality attributes that must be maintained throughout shelf life
    - Regulatory or standard requirements driving the study design
    - Material change considerations (old vs new material properties)
    - Sterilization impact considerations on product stability
    - Storage and handling conditions that affect product performance
    - End-user application requirements that influence acceptance criteria
    - Manufacturing variability considerations
    - Statistical confidence requirements
    - Timeline constraints and business considerations
    - Safety and efficacy considerations throughout product lifecycle
    - Any assumptions or limitations acknowledged in the study approach
    """
def get_section_5_instructions():
    return """
    ## Core Requirements
    - Use ONLY information from the provided V&V document - no external knowledge or invented details
    - Extract information about preparation steps required before executing stability tests
    - Focus on practical setup and preparation activities
    
    ### 5. Testing Preparations
    - Sample selection criteria and batch identification
    - Sample quantity requirements for each test group
    - Sample packaging and labeling requirements
    - Equipment calibration and qualification needs
    - Environmental chamber setup and validation (temperature, humidity)
    - Sterilization preparation (gamma irradiation, ETO cycles)
    - Documentation preparation (test forms, data sheets, protocols)
    - Personnel training and qualification requirements
    - Material handling and storage procedures before testing
    - Test fixture and tooling preparation
    - Baseline measurements and initial characterization
    - Timeline for preparation activities
    - Quality control checks before test initiation
    - Storage conditions for samples awaiting testing
    - Traceability and chain of custody procedures
    - Any special handling requirements for the new silicone material
    """

def get_section_6_instructions():
    return """
    ## Core Requirements
    - Use ONLY information from the provided V&V document - no external knowledge or invented details
    - Extract specific test procedures and detailed measurement steps
    - Focus on the actual execution of stability tests with step-by-step details
    
    ### 6. Test Details
    - Detailed test procedures for each functional parameter measured
    - Specific measurement techniques and equipment used
    - Test sequence and order of operations
    - Sampling procedures and handling during testing
    - Data collection methods and recording requirements
    - Quality control checks during testing
    - Environmental conditions during actual testing
    - Test setup and configuration details
    - Measurement accuracy and precision requirements
    - Pass/fail criteria for each test parameter
    - Retest procedures if initial results are questionable
    - Documentation requirements during testing
    - Safety procedures during test execution
    - Time constraints and testing windows
    - Specific test points and intervals (1 month, 2 years, 5 years equivalent)
    - Post-sterilization testing procedures
    - Visual inspection criteria and methods
    - Dimensional measurement procedures
    - Functional performance test details
    - Data verification and review procedures
    """

In [52]:
def get_openai_response(file, query):
    client = OpenAI()

    completion = client.chat.completions.create(
    model="o3",
    reasoning_effort="high",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "file",
                    "file": {
                        "file_id": file.id,
                    }
                },
                {
                    "type": "text",
                    "text": query,
                },
            ]
        }
    ]
)

    return completion.choices[0].message.content

In [53]:
def load_file_to_openai(file_path):
    client = OpenAI()
    file = client.files.create(
        file=open(file_path, "rb"),
        purpose="user_data"
    )
    return file

In [49]:
file = load_file_to_openai("docs/V&V Report.pdf")

In [50]:
file

FileObject(id='file-TmPaarwRhNnHHh12ddMqep', bytes=741819, created_at=1749221258, filename='V&V Report.pdf', object='file', purpose='user_data', status='processed', expires_at=None, status_details=None)

In [57]:
response1= get_openai_response(file, get_section_1_2_instruction())
print("Section 1 and 2 Response:")
print(response1)
response2= get_openai_response(file, get_section_3_instructions())
print("Section 3 Response:")
print(response2)
response3= get_openai_response(file, get_section_4_instructions())
print("Section 4 Response:")
print(response3)
response4= get_openai_response(file, get_section_5_instructions())
print("Section 5 Response:")
print(response4)
response5= get_openai_response(file, get_section_6_instructions())
print("Section 6 Response:")
print(response5)

Section 1 and 2 Response:
1. Summary (Executive Summary)  
The stability evaluation described in “Silicon Change on NIP valve – Design Verification & Validation Report, Doc-331335, Rev. A, 06-Apr-2025” was performed to confirm that replacing the silicone septum material with SILPURAN® 6000/50 A+B (septum PN 590422) does not compromise long-term performance of Elcam’s Stand-Alone NIP Luer-Activated Valve (product PN 220808L) and associated devices.

Accelerated-aging regimens (section 4.7) were chosen to simulate real-time storage of up to five years.  Test groups were exposed to:
• 52 °C for 7, 98 and 245 days (≈ 1 month, 2 years, 5 years)  
• 60 °C for 29, 57 and 141 days (≈ 1 year, 2 years, 5 years)  

The most severe configuration (Group F5-5-60-EG: 40 kGy Gamma, 2×EtO, 141 days @ 60 °C = 5 years) was selected as the primary stability challenge.  Functional, dimensional and safety tests defined in AAMI ISO CN27-2021 were executed after aging and sterilisation.  No degradation or non

Section 3 Response:
Filtered references – Stability / Shelf-Life relevant  

1. Doc-3313350 A – “Design Verification & Validation Plan”  
   • Defines overall V&V methodology, including accelerated-aging and real-time stability studies used in this report.  

2. Doc-310150 B – “PRJ-0695 – Silicon Change on NIP valve – Design Verification & Validation Plan”  
   • Project-specific test plan that details the aging, sterilization and shelf-life evaluation strategy for the new silicone septum.  

3. Doc-310418 C – “Product Risk Management Document (FMEA)”  
   • Identifies aging- and sterilization-related failure modes (e.g., seal degradation, material embrittlement) and drives the need for stability testing.  

4. Doc-301367 01 – “ECO-24-0005 – PN 590408 Silicone Change”  
   • Engineering Change Order that triggered the material substitution; establishes the requirement to repeat aging / stability studies.  

5. Doc-332257 A – “Product Stability Test Protocol”  
   • Primary protocol des